In [1]:
%matplotlib inline
import os
import sys
sys.path.append("./models/")
sys.path.append("./models/object_detection")

import time

import numpy as np
from matplotlib import pyplot as plt
import cv2
import skimage.io as io
import tensorflow as tf

from utils import label_map_util
from utils import visualization_utils as vis_util

In [2]:
PATH_TO_CKPT = os.path.join('ssd_mobilenet_v1_coco_11_06_2017', 'frozen_inference_graph.pb')
PATH_TO_LABELS = os.path.join('models', 'object_detection', 'data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90

In [3]:
# Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [4]:
# Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [5]:
TEST_IMAGE_PATHS = [ os.path.join('models', 'object_detection', 'test_images', 'image{}.jpg'.format(i)) for i in range(1, 3) ]
#TEST_IMAGE_PATHS = ["/var/tmp/_3e11d8d9-fb61-4f25-abaa-b402dad002dc_"+str(1000000+i)[1:]+".jpg" for i in range(1,31)]

In [6]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:

            start = time.time()
            image_np = io.imread(image_path).astype(np.uint8)
            print(image_np.shape)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            print("predict", time.time() - start, "sec")


(636, 1024, 3)
predict 1.0898220539093018 sec
(900, 1352, 3)
predict 0.3994472026824951 sec


In [13]:
print(boxes.shape)
boxes = np.squeeze(boxes)
print(boxes.shape)
classes = np.squeeze(classes).astype(np.int32)
scores = np.squeeze(scores)
boxes_list = [tuple(boxes[i].tolist()) for i in range(boxes.shape[0])]
result_list = []
for i, box in enumerate(boxes_list):
        class_name = category_index[classes[i]]['name'] if classes[i] in category_index.keys() else 'N/A'
        percentage = int(100*scores[i])
        # ymin, xmin, ymax, xmax = box
        result_list.append({
            "class_name": class_name,
            "percentage": percentage,
            "box": box,
        })
result_list

(100, 4)
(100, 4)


[{'box': (0.3893212676048279,
   0.3482138216495514,
   0.40933746099472046,
   0.36333873867988586),
  'class_name': 'kite',
  'percentage': 93},
 {'box': (0.5749697089195251,
   0.06333530694246292,
   0.6149569153785706,
   0.07912314683198929),
  'class_name': 'person',
  'percentage': 84},
 {'box': (0.6778091192245483,
   0.07910311967134476,
   0.8387483358383179,
   0.12358539551496506),
  'class_name': 'person',
  'percentage': 75},
 {'box': (0.08469407260417938,
   0.43690937757492065,
   0.1742096096277237,
   0.4994242787361145),
  'class_name': 'kite',
  'percentage': 75},
 {'box': (0.5817533731460571,
   0.025888990610837936,
   0.6209155321121216,
   0.03864923492074013),
  'class_name': 'person',
  'percentage': 73},
 {'box': (0.5604797601699829,
   0.3907437026500702,
   0.5917688608169556,
   0.40679821372032166),
  'class_name': 'person',
  'percentage': 68},
 {'box': (0.2703571915626526,
   0.20379848778247833,
   0.3090563416481018,
   0.2272176295518875),
  'class_